In [6]:
!ls

'Lesson 1-Stripped [Code only].ipynb'  'Lesson 5-Stripped [Code Only].ipynb'
'Lesson 2-Stripped[Code only].ipynb'    aclImdb
'Lesson 3-Stripped [Code only].ipynb'   aclImdb_v1.tar
'Lesson 4-Stripped [Code only].ipynb'   tmp


In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [44]:
#from fastai.nlp import *
import os
import re
from glob import glob
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

In [35]:
PATH = "./aclImdb/"
names = ['neg','pos']

In [36]:
def texts_labels_from_folders(path, folders):
    texts,labels = [],[]
    for idx,label in enumerate(folders):
        for fname in glob(os.path.join(path, label, '*.*')):
            texts.append(open(fname, 'r').read())
            labels.append(idx)
    return texts, np.array(labels).astype(np.int64)

In [37]:
trn,trn_y = texts_labels_from_folders(f'{PATH}train',names)
val,val_y = texts_labels_from_folders(f'{PATH}test',names)

In [38]:
trn[0]

"College students, who are clearing out a condemned dormitory, are stalked by an elusive killer.<br /><br />The Dorm That Dripped Blood (aka Pranks) is a bit of a mixed bag for slasher fans. The movies production values are pretty low and the story for the most part is pretty routine, there's even a creepy bum hanging around for a red herring. In fact much of the story's build-up is pretty forgettable, save for one or two brutal murders. But the movie is really made better by its surprisingly intense climax (in an atmospheric setting) and one fairly bold, unconventional conclusion.<br /><br />The cast is lackluster for the most part. Stephen Sachs is the best of the lot as he does a pretty nice turn in character. Also look for a young Daphne Zuniga as an ill-fated student.<br /><br />Over all this is a pretty standard B slasher effort, but the finale is well worth savoring and for this viewer saved the movie from being a complete ho-hum.<br /><br />** out of ****"

In [39]:
trn_y[0]

0

In [46]:
re_tok = re.compile('([!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~“”¨«»®´·º½¾¿¡§£₤‘’])')

In [47]:
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [48]:
veczr = CountVectorizer(tokenizer=tokenize)

In [49]:
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

In [50]:
trn_term_doc

<25000x75132 sparse matrix of type '<class 'numpy.int64'>'
	with 3749745 stored elements in Compressed Sparse Row format>

In [51]:
trn_term_doc[0]

<1x75132 sparse matrix of type '<class 'numpy.int64'>'
	with 123 stored elements in Compressed Sparse Row format>

In [52]:
vocab = veczr.get_feature_names(); vocab[5000:5005]

['aussie', 'aussies', 'austen', 'austeniana', 'austens']

In [53]:
w0 = set([o.lower() for o in trn[0].split(' ')]); w0

{'(aka',
 '(in',
 '****',
 '/>**',
 '/><br',
 '/>over',
 '/>the',
 'a',
 'all',
 'also',
 'an',
 'and',
 'are',
 'around',
 'as',
 'atmospheric',
 'b',
 'bag',
 'being',
 'best',
 'better',
 'bit',
 'blood',
 'bold,',
 'brutal',
 'build-up',
 'bum',
 'but',
 'by',
 'cast',
 'character.',
 'clearing',
 'climax',
 'college',
 'complete',
 'conclusion.<br',
 'condemned',
 'creepy',
 'daphne',
 'does',
 'dorm',
 'dormitory,',
 'dripped',
 'effort,',
 'elusive',
 'even',
 'fact',
 'fairly',
 'fans.',
 'finale',
 'for',
 'forgettable,',
 'from',
 'hanging',
 'he',
 'herring.',
 'ho-hum.<br',
 'ill-fated',
 'in',
 'intense',
 'is',
 'its',
 'killer.<br',
 'lackluster',
 'look',
 'lot',
 'low',
 'made',
 'mixed',
 'most',
 'movie',
 'movies',
 'much',
 'murders.',
 'nice',
 'of',
 'one',
 'or',
 'out',
 'part',
 'part.',
 'pranks)',
 'pretty',
 'production',
 'really',
 'red',
 'routine,',
 'sachs',
 'save',
 'saved',
 'savoring',
 'setting)',
 'slasher',
 'stalked',
 'standard',
 'stephen',
 

In [54]:
len(w0)

115

In [55]:
veczr.vocabulary_['absurd']

1297

In [56]:
trn_term_doc[0,1297]

0

In [57]:
trn_term_doc[0,5000]

0

In [58]:
def pr(y_i):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [59]:
x=trn_term_doc
y=trn_y

r = np.log(pr(1)/pr(0))
b = np.log((y==1).mean() / (y==0).mean())

In [60]:
pre_preds = val_term_doc @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

0.81656

In [61]:
x=trn_term_doc.sign()
r = np.log(pr(1)/pr(0))

pre_preds = val_term_doc.sign() @ r.T + b
preds = pre_preds.T>0
(preds==val_y).mean()

0.83016

In [64]:
#TODO: Original nb had dual =True, check why
# Note 2: 0.19.2 was the sklearn version there

In [63]:
m = LogisticRegression(C=1e8, )#dual=True)
m.fit(x, y)
preds = m.predict(val_term_doc)
(preds==val_y).mean()

0.83392

In [65]:
m = LogisticRegression(C=1e8)
m.fit(trn_term_doc.sign(), y)
preds = m.predict(val_term_doc.sign())
(preds==val_y).mean()

0.85708

In [71]:
#TODO: In original notebook, this is higher, `0.84x`

In [68]:
m = LogisticRegression(C=0.1)
m.fit(x, y)
preds = m.predict(val_term_doc)
(preds==val_y).mean()

/home/protonsuper/anaconda3/envs/fastai-ml/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7592

In [69]:
m = LogisticRegression(C=0.1)
m.fit(trn_term_doc.sign(), y)
preds = m.predict(val_term_doc.sign())
(preds==val_y).mean()

/home/protonsuper/anaconda3/envs/fastai-ml/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.88432

In [70]:
veczr =  CountVectorizer(ngram_range=(1,3), tokenizer=tokenize, max_features=800000)
trn_term_doc = veczr.fit_transform(trn)
val_term_doc = veczr.transform(val)

In [72]:
trn_term_doc.shape

(25000, 800000)

In [73]:
vocab = veczr.get_feature_names()

In [74]:
vocab[200000:200005]

['by vast', 'by vengeance', 'by vengeance .', 'by vera', 'by vera miles']

In [75]:
y=trn_y
x=trn_term_doc.sign()
val_x = val_term_doc.sign()

In [76]:
r = np.log(pr(1) / pr(0))
b = np.log((y==1).mean() / (y==0).mean())

In [78]:
m = LogisticRegression(C=0.1)
m.fit(x, y);

preds = m.predict(val_x)
(preds.T==val_y).mean()

/home/protonsuper/anaconda3/envs/fastai-ml/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.90472

In [79]:
r.shape, r

((1, 800000),
 matrix([[-0.05468386, -0.16100472, -0.24783616, ...,  1.09861229,
          -0.69314718, -0.69314718]]))

In [80]:
np.exp(r)

matrix([[0.94678442, 0.85128806, 0.7804878 , ..., 3.        , 0.5       ,
         0.5       ]])

In [83]:
x_nb = x.multiply(r)
m = LogisticRegression(C=0.1)
m.fit(x_nb, y);

val_x_nb = val_x.multiply(r)
preds = m.predict(val_x_nb)
(preds.T==val_y).mean()

0.91748

In [84]:
sl=2000

# Note: Need to make things work after this

In [85]:
# Here is how we get a model from a bag of words
md = TextClassifierData.from_bow(trn_term_doc, trn_y, val_term_doc, val_y, sl)

NameError: name 'TextClassifierData' is not defined

In [ ]:
learner = md.dotprod_nb_learner()
learner.fit(0.02, 1, wds=1e-6, cycle_len=1)

In [ ]:
learner.fit(0.02, 2, wds=1e-6, cycle_len=1)

In [ ]:
learner.fit(0.02, 2, wds=1e-6, cycle_len=1)